Разработка основы для создания платформы A/B тестирования (пошагово будем добавлять сервисы)

**A/B платформ состоит из 4 блоков:**

- **DataService** — предоставляет доступ к сырым данным;
- **MetricsService** — вычисляет метрики;
- **ExperimentsService** — оценивает эксперименты;
- **SplittingService** — подбирает группы пользователей для эксперимента.


### Метод get_data_subset класса DataService.

In [1]:
import pandas as pd

from datetime import datetime


class DataService:

    def __init__(self, table_name_2_table):
        """Класс, предоставляющий доступ к сырым данным.
        
        :param table_name_2_table (dict[str, pd.DataFrame]): словарь таблиц с данными.
            Пример, {
                'sales': pd.DataFrame({'sale_id': ['123', ...], ...}),
                ...
            }. 
        """
        self.table_name_2_table = table_name_2_table

    def get_data_subset(self, table_name, begin_date, end_date, user_ids=None, columns=None):
        """Возвращает подмножество данных.

        :param table_name (str): название таблицы с данными.
        :param begin_date (datetime.datetime): дата начала интервала с данными.
            Пример, df[df['date'] >= begin_date].
            Если None, то фильтровать не нужно.
        :param end_date (None, datetime.datetime): дата окончания интервала с данными.
            Пример, df[df['date'] < end_date].
            Если None, то фильтровать не нужно.
        :param user_ids (None, list[str]): список user_id, по которым нужно предоставить данные.
            Пример, df[df['user_id'].isin(user_ids)].
            Если None, то фильтровать по user_id не нужно.
        :param columns (None, list[str]): список названий столбцов, по которым нужно предоставить данные.
            Пример, df[columns].
            Если None, то фильтровать по columns не нужно.

        :return df (pd.DataFrame): датафрейм с подмножеством данных.
        """
        df = self.table_name_2_table[table_name]
        if begin_date:
            df = df[df['date'] >= begin_date]
        if end_date:
            df = df[df['date'] < end_date]
        if user_ids:
            df = df[df['user_id'].isin(user_ids)]
        if columns:
            df = df[columns]
        return df.copy()


def _chech_df(df, df_ideal, sort_by):
    assert isinstance(df, pd.DataFrame), 'Функция вернула не pd.DataFrame.'
    assert len(df) == len(df_ideal), 'Неверное количество строк.'
    assert len(df.T) == len(df_ideal.T), 'Неверное количество столбцов.'
    columns = df_ideal.columns
    assert df.columns.isin(columns).sum() == len(df.columns), 'Неверное название столбцов.'
    df = df[columns].sort_values(sort_by)
    df_ideal = df_ideal.sort_values(sort_by)
    assert df_ideal.equals(df), 'Итоговый датафрейм не совпадает с верным результатом.'


if __name__ == '__main__':
    table = pd.DataFrame({
        'date': [datetime(2022, 1, 5, 12,), datetime(2022, 1, 7, 12)],
        'user_id': ['1', '2'],
    })
    ideal_df = pd.DataFrame({
        'date': [datetime(2022, 1, 5, 12,)],
        'user_id': ['1'],
    })

    data_service = DataService({'table': table})
    res_df = data_service.get_data_subset('table', datetime(2022, 1, 1), datetime(2022, 1, 6))
    _chech_df(res_df, ideal_df, 'date')
    print('simple test passed')


simple test passed


### Вычисления метрик «revenue (web)», «revenue (all)», «response time». 

In [2]:
import pandas as pd

from datetime import datetime


class DataService:
    """Сервис для доступа и фильтрации сырых данных.
    
    Предоставляет методы для получения подмножеств данных из различных таблиц
    с возможностью фильтрации по датам, пользователям и столбцам.

    Attributes:
        table_name_2_table (dict[str, pd.DataFrame]): Словарь таблиц с данными,
            где ключ - название таблицы, значение - DataFrame с данными.
    """

    def __init__(self, table_name_2_table):
        """Инициализирует DataService с указанными таблицами данных.
        
        Args:
            table_name_2_table (dict[str, pd.DataFrame]): Словарь таблиц с данными.
                Пример: {
                    'sales': pd.DataFrame({'sale_id': ['123', ...], ...}),
                    'web-logs': pd.DataFrame(...),
                    ...
                }
        """
        self.table_name_2_table = table_name_2_table

    def get_data_subset(self, table_name, begin_date, end_date, user_ids=None, columns=None):
        """Возвращает отфильтрованное подмножество данных из указанной таблицы.
        
        Args:
            table_name (str): Название таблицы для получения данных.
            begin_date (datetime): Начальная дата периода (включительно). 
                Если None, нижняя граница не применяется.
            end_date (datetime): Конечная дата периода (исключительно).
                Если None, верхняя граница не применяется.
            user_ids (list[str], optional): Список user_id для фильтрации.
                Если None, фильтрация по пользователям не выполняется.
            columns (list[str], optional): Список столбцов для включения в результат.
                Если None, возвращаются все столбцы.
        
        Returns:
            pd.DataFrame: Отфильтрованный DataFrame с запрошенными данными.
            
        Note:
            Всегда возвращает копию данных, чтобы избежать неожиданных изменений.
        """
        df = self.table_name_2_table[table_name]
        if begin_date:
            df = df[df['date'] >= begin_date]
        if end_date:
            df = df[df['date'] < end_date]
        if user_ids:
            df = df[df['user_id'].isin(user_ids)]
        if columns:
            df = df[columns]
        return df.copy()


class MetricsService:
    """Сервис для расчета метрик для A/B тестирования.
    
    Обеспечивает вычисление различных метрик на основе данных из DataService.
    Все метрики возвращаются в стандартизированном формате.

    Attributes:
        data_service (DataService): Экземпляр сервиса для доступа к данным.
    """

    def __init__(self, data_service):
        """Инициализирует MetricsService с указанным сервисом данных.
        
        Args:
            data_service (DataService): Сервис для доступа к сырым данным.
        """
        self.data_service = data_service

    def _get_data_subset(self, table_name, begin_date, end_date, user_ids=None, columns=None):
        """Вспомогательный метод для получения подмножества данных.
        
        Прокси-метод для DataService.get_data_subset().
        """
        return self.data_service.get_data_subset(table_name, begin_date, end_date, user_ids, columns)

    def _calculate_response_time(self, begin_date, end_date, user_ids):
        """Вычисляет время отклика сервера для каждого запроса.
        
        Args:
            begin_date (datetime): Начальная дата периода.
            end_date (datetime): Конечная дата периода.
            user_ids (list[str], optional): Список user_id для фильтрации.
        
        Returns:
            pd.DataFrame: DataFrame с колонками:
                - user_id (str): Идентификатор пользователя
                - metric (float): Время загрузки в миллисекундах
        """
        return (
            self._get_data_subset('web-logs', begin_date, end_date, user_ids, ['user_id', 'load_time'])
            .rename(columns={'load_time': 'metric'})
            [['user_id', 'metric']]
        )
    
    def _calculate_revenue_web(self, begin_date, end_date, user_ids):
        """Вычисляет выручку от пользователей, посетивших сайт в указанный период.
        
        Args:
            begin_date (datetime): Начальная дата периода.
            end_date (datetime): Конечная дата периода.
            user_ids (list[str], optional): Список user_id для фильтрации.
        
        Returns:
            pd.DataFrame: DataFrame с колонками:
                - user_id (str): Идентификатор пользователя
                - metric (float): Суммарная выручка от пользователя
                
        Note:
            Для пользователей без покупок выручка устанавливается в 0.
            Включает только пользователей, посетивших сайт в указанный период.
        """
        user_ids_ = (
            self._get_data_subset('web-logs', begin_date, end_date, user_ids, ['user_id'])
            ['user_id'].unique()
        )

        df = (
            self._get_data_subset('sales', begin_date, end_date, user_ids, ['user_id', 'price'])
            .groupby('user_id')[['price']].sum().reset_index()
            .rename(columns={'price': 'metric'})
        )
        df = pd.merge(pd.DataFrame({'user_id': user_ids_}), df, on='user_id', how='left').fillna(0)
        return df[['user_id', 'metric']]
    
    def _calculate_revenue_all(self, begin_date, end_date, user_ids):
        """Вычисляет выручку от всех пользователей, когда-либо посещавших сайт.
        
        Args:
            begin_date (datetime): Начальная дата периода.
            end_date (datetime): Конечная дата периода.
            user_ids (list[str], optional): Список user_id для фильтрации.
        
        Returns:
            pd.DataFrame: DataFrame с колонками:
                - user_id (str): Идентификатор пользователя
                - metric (float): Суммарная выручка от пользователя
                
        Note:
            Для пользователей без покупок выручка устанавливается в 0.
            Включает всех пользователей, посещавших сайт до end_date.
        """
        user_ids_ = (
            self._get_data_subset('web-logs', None, end_date, user_ids, ['user_id'])
            ['user_id'].unique()
        )
        df = (
            self._get_data_subset('sales', begin_date, end_date, user_ids, ['user_id', 'price'])
            .groupby('user_id')[['price']].sum().reset_index()
            .rename(columns={'price': 'metric'})
        )
        df = pd.merge(pd.DataFrame({'user_id': user_ids_}), df, on='user_id', how='left').fillna(0)
        return df[['user_id', 'metric']]
    
    def calculate_metric(self, metric_name, begin_date, end_date, user_ids=None):
        """Вычисляет указанную метрику за заданный период.
        
        Args:
            metric_name (str): Название метрики. Доступные значения:
                - 'response time': Время отклика сервера
                - 'revenue (web)': Выручка от посетителей сайта
                - 'revenue (all)': Выручка от всех пользователей
            begin_date (datetime): Начальная дата периода (включительно).
            end_date (datetime): Конечная дата периода (исключительно).
            user_ids (list[str], optional): Список user_id для фильтрации.
                Если None, вычисляет для всех пользователей.
        
        Returns:
            pd.DataFrame: DataFrame с колонками ['user_id', 'metric']
            
        Raises:
            ValueError: Если передано неизвестное имя метрики.
        """
        match metric_name:
            case 'response time':
                return self._calculate_response_time(begin_date, end_date, user_ids)
            case 'revenue (web)':
                return self._calculate_revenue_web(begin_date, end_date, user_ids)
            case 'revenue (all)':
                return self._calculate_revenue_all(begin_date, end_date, user_ids)
            case _:
                raise ValueError(f'Unknown metric name: {metric_name}')
            
def _chech_df(df, df_ideal, sort_by, reindex=False, set_dtypes=False):
    assert isinstance(df, pd.DataFrame), 'Функция вернула не pd.DataFrame.'
    assert len(df) == len(df_ideal), 'Неверное количество строк.'
    assert len(df.T) == len(df_ideal.T), 'Неверное количество столбцов.'
    columns = df_ideal.columns
    assert df.columns.isin(columns).sum() == len(df.columns), 'Неверное название столбцов.'
    df = df[columns].sort_values(sort_by)
    df_ideal = df_ideal.sort_values(sort_by)
    if reindex:
        df_ideal.index = range(len(df_ideal))
        df.index = range(len(df))
    if set_dtypes:
        for column, dtype in df_ideal.dtypes.to_dict().items():
            df[column] = df[column].astype(dtype)
    assert df_ideal.equals(df), 'Итоговый датафрейм не совпадает с верным результатом.'


if __name__ == '__main__':
    df_sales = pd.DataFrame({
        'sale_id': [1, 2, 3],
        'date': [datetime(2022, 3, day, 11) for day in range(11, 14)],
        'price': [1100, 900, 1500],
        'user_id': ['1', '2', '1'],
    })
    df_web_logs = pd.DataFrame({
        'date': [datetime(2022, 3, day, 11) for day in range(10, 14)],
        'load_time': [80.8, 90.1, 15.8, 19.7],
        'user_id': ['3', '1', '2', '1'],
    })
    begin_date = datetime(2022, 3, 11, 9)
    end_date = datetime(2022, 4, 11, 9)

    ideal_response_time = pd.DataFrame({'user_id': ['1', '2', '1'], 'metric': [90.1, 15.8, 19.7],})
    ideal_revenue_web = pd.DataFrame({'user_id': ['1', '2'], 'metric': [2600., 900.],})
    ideal_revenue_all = pd.DataFrame({'user_id': ['1', '2', '3'], 'metric': [2600., 900., 0.],})

    data_service = DataService({'sales': df_sales, 'web-logs': df_web_logs})
    metrics_service = MetricsService(data_service)

    df_response_time = metrics_service.calculate_metric('response time', begin_date, end_date)
    df_revenue_web = metrics_service.calculate_metric('revenue (web)', begin_date, end_date)
    df_revenue_all = metrics_service.calculate_metric('revenue (all)', begin_date, end_date)

    _chech_df(df_response_time, ideal_response_time, ['user_id', 'metric'], True, True)
    _chech_df(df_revenue_web, ideal_revenue_web, ['user_id', 'metric'], True, True)
    _chech_df(df_revenue_all, ideal_revenue_all, ['user_id', 'metric'], True, True)
    print('simple test passed')

        
          

simple test passed
